<a href="https://colab.research.google.com/github/Chthanh/Recommender-System/blob/main/User-based%20Collaborative%20Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Load dataset
data = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/RecSys/data_01.xls")
data.rename(columns={"Unnamed: 0": "userId"}, inplace=True)
data.set_index('userId', inplace = True)
data.head()

,11: Star Wars: Episode IV - A New Hope (1977),12: Finding Nemo (2003),13: Forrest Gump (1994),14: American Beauty (1999),22: Pirates of the Caribbean: The Curse of the Black Pearl (2003),24: Kill Bill: Vol. 1 (2003),38: Eternal Sunshine of the Spotless Mind (2004),63: Twelve Monkeys (a.k.a. 12 Monkeys) (1995),77: Memento (2000),85: Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981),...,8467: Dumb & Dumber (1994),8587: The Lion King (1994),9331: Clear and Present Danger (1994),9741: Unbreakable (2000),9802: The Rock (1996),9806: The Incredibles (2004),10020: Beauty and the Beast (1991),36657: X-Men (2000),36658: X2: X-Men United (2003),36955: True Lies (1994)
userId,,,,,,,,,,,,,,,,,,,,,
1648,NaN,NaN,NaN,NaN,4.0,3.0,NaN,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,5.0,3.5,3.0,NaN,3.5,NaN
5136,4.5,5.0,5.0,4.0,5.0,5.0,5.0,3.0,NaN,5.0,...,1.0,5.0,NaN,NaN,NaN,5.0,5.0,4.5,4.0,NaN
918,5.0,5.0,4.5,NaN,3.0,NaN,5.0,NaN,5.0,NaN,...,NaN,5.0,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN
2824,4.5,NaN,5.0,NaN,4.5,4.0,NaN,NaN,5.0,NaN,...,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3867,4.0,4.0,4.5,NaN,4.0,3.0,NaN,NaN,NaN,4.5,...,1.0,4.0,NaN,NaN,NaN,3.0,4.0,4.0,3.5,3.0


In [ ]:
# print dataset shape
data.shape

(25, 100)

In [ ]:
def pearson_corr(user_u, user_v):
  u_avg = np.nansum(user_u)/np.count_nonzero(~np.isnan(user_u))
  v_avg = np.nansum(user_v)/np.count_nonzero(~np.isnan(user_v))
  mutual_index = [i for i in range(len(user_u)) if not np.isnan(user_u[i]) and not np.isnan(user_v[i])]

  dividend = 0
  u_divisor = 0
  v_divisor = 0

  for index in mutual_index:
    u_nr = user_u[index] - u_avg
    v_nr = user_v[index] - v_avg
    dividend += u_nr * v_nr
    u_divisor += pow(u_nr, 2)
    v_divisor += pow(v_nr, 2)

  divisor = np.sqrt(u_divisor) * np.sqrt(v_divisor)

  if divisor != 0:
    return round(dividend / divisor, 3)

  return

In [ ]:
def corr_matrix(data):
  m, n = data.shape
  corr_mx = np.identity(m)
  for i in range(m-1):
    for j in range(i+1,m):
      corr_mx[i,j] = corr_mx[j,i] = pearson_corr(data.iloc[i].tolist(), data.iloc[j].tolist())
  corr_mx = pd.DataFrame(corr_mx, index=data.index, columns=data.index)

  return corr_mx

In [ ]:
  pearson_mx = corr_matrix(data.head())
  pearson_mx

userId,1648,5136,918,2824,3867
userId,,,,,
1648,1.000,0.376,-0.209,0.513,0.303
5136,0.376,1.000,0.107,-0.005,0.342
918,-0.209,0.107,1.000,-0.270,0.296
2824,0.513,-0.005,-0.270,1.000,-0.091
3867,0.303,0.342,0.296,-0.091,1.000


In [ ]:
pearson_mx.drop(index=1648, inplace=True)
pearson_mx

userId,1648,5136,918,2824,3867
userId,,,,,
5136,0.376,1.000,0.107,-0.005,0.342
918,-0.209,0.107,1.000,-0.270,0.296
2824,0.513,-0.005,-0.270,1.000,-0.091
3867,0.303,0.342,0.296,-0.091,1.000


In [ ]:
threshold = 0.2
pearson_mx[1648]> threshold

userId
5136     True
918     False
2824     True
3867     True
Name: 1648, dtype: bool

In [ ]:
pearson_mx[pearson_mx[1648]>threshold]

userId,1648,5136,918,2824,3867
userId,,,,,
5136,0.376,1.000,0.107,-0.005,0.342
2824,0.513,-0.005,-0.270,1.000,-0.091
3867,0.303,0.342,0.296,-0.091,1.000


In [ ]:
pearson_mx[pearson_mx[1648]>threshold][1648]

userId
5136    0.376
2824    0.513
3867    0.303
Name: 1648, dtype: float64

In [ ]:
pearson_mx[pearson_mx[1648]>threshold][1648].sort_values(ascending=False)[:2]

userId
2824    0.513
5136    0.376
Name: 1648, dtype: float64

In [ ]:
similar_users = pearson_mx[pearson_mx[1648]>threshold][1648].sort_values(ascending=False)[:2]

In [ ]:
data[data.index.isin(similar_users.index)].dropna(axis = 1 , how = "all")

,11: Star Wars: Episode IV - A New Hope (1977),12: Finding Nemo (2003),13: Forrest Gump (1994),14: American Beauty (1999),22: Pirates of the Caribbean: The Curse of the Black Pearl (2003),24: Kill Bill: Vol. 1 (2003),38: Eternal Sunshine of the Spotless Mind (2004),63: Twelve Monkeys (a.k.a. 12 Monkeys) (1995),77: Memento (2000),85: Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981),...,4327: Charlie's Angels (2000),5503: The Fugitive (1993),7443: Chicken Run (2000),8358: Cast Away (2000),8467: Dumb & Dumber (1994),8587: The Lion King (1994),9806: The Incredibles (2004),10020: Beauty and the Beast (1991),36657: X-Men (2000),36658: X2: X-Men United (2003)
userId,,,,,,,,,,,,,,,,,,,,,
5136,4.5,5.0,5.0,4.0,5.0,5.0,5.0,3.0,NaN,5.0,...,1.5,4.0,1.0,4.0,1.0,5.0,5.0,5.0,4.5,4.0
2824,4.5,NaN,5.0,NaN,4.5,4.0,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN


In [ ]:
similar_user_rated = data[data.index.isin(similar_users.index)].dropna(axis = 1 , how = "all")

In [ ]:
data[data.index == 1648].dropna(axis=1, how='all').columns

Index(['22: Pirates of the Caribbean: The Curse of the Black Pearl (2003)',
       '24: Kill Bill: Vol. 1 (2003)', '105: Back to the Future (1985)',
       '146: Crouching Tiger Hidden Dragon (Wo hu cang long) (2000)',
       '155: The Dark Knight (2008)', '161: Ocean's Eleven (2001)',
       '180: Minority Report (2002)', '272: Batman Begins (2005)',
       '280: Terminator 2: Judgment Day (1991)', '329: Jurassic Park (1993)',
       '393: Kill Bill: Vol. 2 (2004)', '414: Batman Forever (1995)',
       '424: Schindler's List (1993)', '453: A Beautiful Mind (2001)',
       '557: Spider-Man (2002)', '558: Spider-Man 2 (2004)',
       '568: Apollo 13 (1995)', '597: Titanic (1997)',
       '602: Independence Day (a.k.a. ID4) (1996)', '603: The Matrix (1999)',
       '604: The Matrix Reloaded (2003)',
       '607: Men in Black (a.k.a. MIB) (1997)',
       '640: Catch Me If You Can (2002)',
       '672: Harry Potter and the Chamber of Secrets (2002)',
       '862: Toy Story (1995)', '954: M

In [ ]:
target_user_rated = data[data.index == 1648].dropna(axis=1, how='all')

In [ ]:
data[data.index == 1648].dropna(axis=1, how='all')

,22: Pirates of the Caribbean: The Curse of the Black Pearl (2003),24: Kill Bill: Vol. 1 (2003),105: Back to the Future (1985),146: Crouching Tiger Hidden Dragon (Wo hu cang long) (2000),155: The Dark Knight (2008),161: Ocean's Eleven (2001),180: Minority Report (2002),272: Batman Begins (2005),280: Terminator 2: Judgment Day (1991),329: Jurassic Park (1993),...,672: Harry Potter and the Chamber of Secrets (2002),862: Toy Story (1995),954: Mission: Impossible (1996),955: Mission: Impossible II (2000),1894: Star Wars: Episode II - Attack of the Clones (2002),8587: The Lion King (1994),9802: The Rock (1996),9806: The Incredibles (2004),10020: Beauty and the Beast (1991),36658: X2: X-Men United (2003)
userId,,,,,,,,,,,,,,,,,,,,,
1648,4.0,3.0,4.5,2.5,5.0,4.0,3.0,5.0,3.5,4.5,...,3.0,4.0,4.5,4.0,3.0,4.0,5.0,3.5,3.0,3.5


In [ ]:
similar_user_rated.drop(target_user_rated.columns, axis=1, inplace=True, errors='ignore')

In [ ]:
similar_user_rated

,11: Star Wars: Episode IV - A New Hope (1977),12: Finding Nemo (2003),13: Forrest Gump (1994),14: American Beauty (1999),38: Eternal Sunshine of the Spotless Mind (2004),63: Twelve Monkeys (a.k.a. 12 Monkeys) (1995),77: Memento (2000),85: Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981),98: Gladiator (2000),107: Snatch (2000),...,1900: Traffic (2000),2501: The Bourne Identity (2002),2502: The Bourne Supremacy (2004),3049: Ace Ventura: Pet Detective (1994),4327: Charlie's Angels (2000),5503: The Fugitive (1993),7443: Chicken Run (2000),8358: Cast Away (2000),8467: Dumb & Dumber (1994),36657: X-Men (2000)
userId,,,,,,,,,,,,,,,,,,,,,
5136,4.5,5.0,5.0,4.0,5.0,3.0,NaN,5.0,3.5,NaN,...,1.0,4.5,4.0,3.0,1.5,4.0,1.0,4.0,1.0,4.5
2824,4.5,NaN,5.0,NaN,NaN,NaN,5.0,NaN,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def filtering(data, target_user, k, threshold=0.2):
  pearson_mx = corr_matrix(data)

  pearson_mx.drop(index=target_user, inplace=True)

  similar_users = (pearson_mx[pearson_mx[target_user]>threshold][target_user].sort_values(ascending=False)[:k])

  similar_user_rated = data[data.index.isin(similar_users.index)].dropna(axis=1, how='all')

  target_user_rated = data[data.index == target_user].dropna(axis=1, how='all')
  similar_user_rated.drop(target_user_rated.columns, axis=1, inplace=True, errors='ignore')

  return (similar_users, similar_user_rated)

In [ ]:
data[data.index == 89].T.mean()

userId
89    4.397436
dtype: float64

In [ ]:
similar_user_rated['14: American Beauty (1999)']

userId
3712    4.5
2492    3.5
2486    3.5
3853    NaN
2288    NaN
Name: 14: American Beauty (1999), dtype: float64

In [ ]:
similar_user_rated

,14: American Beauty (1999),38: Eternal Sunshine of the Spotless Mind (2004),63: Twelve Monkeys (a.k.a. 12 Monkeys) (1995),77: Memento (2000),105: Back to the Future (1985),107: Snatch (2000),134: O Brother Where Art Thou? (2000),141: Donnie Darko (2001),146: Crouching Tiger Hidden Dragon (Wo hu cang long) (2000),153: Lost in Translation (2003),...,955: Mission: Impossible II (2000),1900: Traffic (2000),2024: The Patriot (2000),2164: Stargate (1994),5503: The Fugitive (1993),7443: Chicken Run (2000),8358: Cast Away (2000),9331: Clear and Present Danger (1994),9741: Unbreakable (2000),9802: The Rock (1996)
userId,,,,,,,,,,,,,,,,,,,,,
3712,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,4.0,...,4.0,NaN,4.5,NaN,4.0,4.0,NaN,NaN,NaN,4.0
2492,3.5,NaN,4.0,4.0,3.5,NaN,NaN,NaN,3.5,4.0,...,3.0,NaN,2.5,4.0,4.0,NaN,NaN,3.0,NaN,3.5
2486,3.5,3.0,3.5,4.5,4.5,3.0,2.0,2.0,4.0,4.0,...,2.0,3.0,0.5,3.0,2.5,2.5,2.5,0.5,3.0,2.5
3853,NaN,5.0,4.0,4.5,NaN,NaN,NaN,4.0,NaN,4.5,...,2.5,4.5,NaN,3.0,2.0,NaN,NaN,NaN,NaN,NaN
2288,NaN,NaN,5.0,5.0,2.0,3.0,NaN,NaN,4.0,1.5,...,3.5,NaN,3.0,2.5,4.0,3.0,4.0,NaN,NaN,4.0


In [ ]:
def pred_score(data, target_user, similar_users, similar_user_rated):
  avg_rating = data[data.index == target_user].T.mean()[target_user]

  pred_score = {}

  for i in similar_user_rated.columns:
    item_rating = similar_user_rated[i]

    nominator = 0
    denominator = 0
    for u in similar_users.index:
      if pd.isna(item_rating[u]) == False:
        user_avg = data[data.index == u].T.mean()[u]
        score = similar_users[u] * (item_rating[u]-user_avg)
        nominator += score
        denominator += similar_users[u]

    pred_score[i] = avg_rating + (nominator / denominator)

  pred_score = (pd.DataFrame(pred_score.items(), columns=['item', 'pred_score'])
                      .sort_values(by='pred_score', ascending=False))

  return pred_score

In [ ]:
def pred_score_nonormalize(data, target_user, similar_users, similar_user_rated):
  pred_score = {}

  for i in similar_user_rated.columns:
    item_rating = similar_user_rated[i]

    nominator = 0
    denominator = 0
    for u in similar_users.index:
      if pd.isna(item_rating[u]) == False:
        score = similar_users[u] * item_rating[u]
        nominator += score
        denominator += similar_users[u]

    pred_score[i] = nominator / denominator

  pred_score = (pd.DataFrame(pred_score.items(), columns=['item', 'pred_score'])
                      .sort_values(by='pred_score', ascending=False))

  return pred_score

**TH1: Không chuẩn hóa dữ liệu**

In [ ]:
# Correlation Matrix
print("Correlation Matrix: ")
corr_matrix(data)

Correlation Matrix: 


userId,1648,5136,918,2824,3867,860,3712,2968,3525,4323,...,3556,5261,2492,5062,2486,4942,2267,4809,3853,2288
userId,,,,,,,,,,,,,,,,,,,,,
1648,1.000,0.376,-0.209,0.513,0.303,0.424,-0.325,0.291,0.090,0.097,...,-0.193,0.487,0.358,0.523,0.003,0.125,-0.427,0.394,-0.333,0.214
5136,0.376,1.000,0.107,-0.005,0.342,0.239,0.126,0.206,0.358,0.038,...,0.488,0.322,0.414,0.225,0.305,0.037,0.249,0.399,0.182,0.390
918,-0.209,0.107,1.000,-0.270,0.296,0.476,0.080,-0.045,0.360,-0.035,...,0.374,0.456,0.080,-0.050,0.134,0.015,-0.595,0.014,0.668,0.077
2824,0.513,-0.005,-0.270,1.000,-0.091,-0.011,0.098,0.203,0.169,0.125,...,-0.231,0.233,0.247,0.265,0.245,0.149,-0.345,0.477,-0.143,0.244
3867,0.303,0.342,0.296,-0.091,1.000,0.283,0.393,0.264,0.124,-0.333,...,0.172,0.284,0.475,0.291,0.439,-0.166,-0.273,0.052,0.431,0.377
860,0.424,0.239,0.476,-0.011,0.283,1.000,0.170,0.075,0.387,0.146,...,0.340,0.401,0.202,0.314,0.276,0.081,0.278,0.165,0.148,0.284
3712,-0.325,0.126,0.080,0.098,0.393,0.170,1.000,0.068,0.096,-0.279,...,0.016,-0.289,-0.115,0.250,0.167,0.146,0.005,-0.463,0.175,0.112
2968,0.291,0.206,-0.045,0.203,0.264,0.075,0.068,1.000,0.029,-0.074,...,0.050,-0.008,0.204,0.033,0.138,0.071,0.109,-0.083,-0.088,0.229
3525,0.090,0.358,0.360,0.169,0.124,0.387,0.096,0.029,1.000,0.211,...,0.476,0.307,0.133,0.300,0.143,0.056,0.177,0.283,0.162,0.193


In [ ]:
# Top 5 users are closest to target user 3867
k = 5
target_user = 3867
similar_users, similar_3867_rated = filtering(data, target_user, k, 0.2)
display(similar_users)

userId
2492    0.475
2486    0.439
3853    0.431
3712    0.393
2288    0.377
Name: 3867, dtype: float64

In [ ]:
# Similar users predict score which target user 3867 have ever rated
predict_score = pred_score_nonormalize(data, target_user, similar_users,similar_3867_rated)
predict_score

,item,pred_score
3,77: Memento (2000),4.471545
23,807: Seven (a.k.a. Se7en) (1995),4.327185
19,629: The Usual Suspects (1995),4.152744
11,194: Amelie (2001),4.133457
8,146: Crouching Tiger Hidden Dragon (Wo hu cang...,4.092340
2,63: Twelve Monkeys (a.k.a. 12 Monkeys) (1995),4.091463
16,424: Schindler's List (1993),4.007317
15,275: Fargo (1996),4.000000
1,38: Eternal Sunshine of the Spotless Mind (2004),3.990805
14,274: The Silence of the Lambs (1991),3.813475


In [ ]:
# Top 5 users are closest to target user 89
k = 5
target_user = 89
similar_users, similar_89_rated = filtering(data, target_user, k, 0.2)
display(similar_users)

userId
4809    0.655
860     0.541
5062    0.512
5136    0.507
3525    0.437
Name: 89, dtype: float64

In [ ]:
# Similar users predict score which target user 3867 have ever rated
predict_score = pred_score_nonormalize(data, target_user, similar_users,similar_89_rated)
predict_score

,item,pred_score
33,807: Seven (a.k.a. Se7en) (1995),4.771495
20,424: Schindler's List (1993),4.731462
9,122: The Lord of the Rings: The Return of the ...,4.699863
7,120: The Lord of the Rings: The Fellowship of ...,4.524038
8,121: The Lord of the Rings: The Two Towers (2002),4.524038
...,...,...
19,414: Batman Forever (1995),2.771380
48,4327: Charlie's Angels (2000),2.503434
16,268: Batman (1989),2.500000
50,7443: Chicken Run (2000),2.479396


**TH2: Chuẩn hóa dữ liệu bằng mean-centering**

In [ ]:
# Correlation Matrix
print("Correlation Matrix: ")
corr_matrix(data)

Correlation Matrix: 


userId,1648,5136,918,2824,3867,860,3712,2968,3525,4323,...,3556,5261,2492,5062,2486,4942,2267,4809,3853,2288
userId,,,,,,,,,,,,,,,,,,,,,
1648,1.000,0.376,-0.209,0.513,0.303,0.424,-0.325,0.291,0.090,0.097,...,-0.193,0.487,0.358,0.523,0.003,0.125,-0.427,0.394,-0.333,0.214
5136,0.376,1.000,0.107,-0.005,0.342,0.239,0.126,0.206,0.358,0.038,...,0.488,0.322,0.414,0.225,0.305,0.037,0.249,0.399,0.182,0.390
918,-0.209,0.107,1.000,-0.270,0.296,0.476,0.080,-0.045,0.360,-0.035,...,0.374,0.456,0.080,-0.050,0.134,0.015,-0.595,0.014,0.668,0.077
2824,0.513,-0.005,-0.270,1.000,-0.091,-0.011,0.098,0.203,0.169,0.125,...,-0.231,0.233,0.247,0.265,0.245,0.149,-0.345,0.477,-0.143,0.244
3867,0.303,0.342,0.296,-0.091,1.000,0.283,0.393,0.264,0.124,-0.333,...,0.172,0.284,0.475,0.291,0.439,-0.166,-0.273,0.052,0.431,0.377
860,0.424,0.239,0.476,-0.011,0.283,1.000,0.170,0.075,0.387,0.146,...,0.340,0.401,0.202,0.314,0.276,0.081,0.278,0.165,0.148,0.284
3712,-0.325,0.126,0.080,0.098,0.393,0.170,1.000,0.068,0.096,-0.279,...,0.016,-0.289,-0.115,0.250,0.167,0.146,0.005,-0.463,0.175,0.112
2968,0.291,0.206,-0.045,0.203,0.264,0.075,0.068,1.000,0.029,-0.074,...,0.050,-0.008,0.204,0.033,0.138,0.071,0.109,-0.083,-0.088,0.229
3525,0.090,0.358,0.360,0.169,0.124,0.387,0.096,0.029,1.000,0.211,...,0.476,0.307,0.133,0.300,0.143,0.056,0.177,0.283,0.162,0.193


In [ ]:
# Top 5 users are closest to target user 3867
k = 5
target_user = 3867
similar_users, similar_3867_rated = filtering(data, target_user, k, 0.2)
display(similar_users)

userId
2492    0.475
2486    0.439
3853    0.431
3712    0.393
2288    0.377
Name: 3867, dtype: float64

In [ ]:
# Similar users predict score which target user 3867 have ever rated
predict_score = pred_score(data, target_user, similar_users,similar_3867_rated)
predict_score

,item,pred_score
3,77: Memento (2000),4.783578
15,275: Fargo (1996),4.771538
23,807: Seven (a.k.a. Se7en) (1995),4.673689
11,194: Amelie (2001),4.451196
2,63: Twelve Monkeys (a.k.a. 12 Monkeys) (1995),4.403496
1,38: Eternal Sunshine of the Spotless Mind (2004),4.361067
8,146: Crouching Tiger Hidden Dragon (Wo hu cang...,4.225583
19,629: The Usual Suspects (1995),4.215289
26,1900: Traffic (2000),4.113366
16,424: Schindler's List (1993),4.069863


In [ ]:
# Top 5 users are closest to target user 89
k = 5
target_user = 89
similar_users, similar_89_rated = filtering(data, target_user, k, 0.2)
display(similar_users)

userId
4809    0.655
860     0.541
5062    0.512
5136    0.507
3525    0.437
Name: 89, dtype: float64

In [ ]:
# Similar users predict score which target user 3867 have ever rated
pred_score(data, 89, similar_users, similar_89_rated)

,item,pred_score
20,424: Schindler's List (1993),5.203526
33,807: Seven (a.k.a. Se7en) (1995),5.200598
9,122: The Lord of the Rings: The Return of the ...,5.192920
7,120: The Lord of the Rings: The Fellowship of ...,5.017096
8,121: The Lord of the Rings: The Two Towers (2002),5.017096
...,...,...
19,414: Batman Forever (1995),3.337694
16,268: Batman (1989),3.183894
48,4327: Charlie's Angels (2000),2.996492
50,7443: Chicken Run (2000),2.972453
